In [1]:
!pip install --upgrade pyswarm
!pip install pymc3
!pip install --upgrade pactools

  Created wheel for pyswarm: filename=pyswarm-0.6-cp36-none-any.whl size=4481 sha256=db786095e3ad78a1c6817d8577981fd1ac6c24f5246f41246ce0878a719fc20f
  Stored in directory: /root/.cache/pip/wheels/37/c5/f6/b33b9ac00040cb95c1f00af982a4197334a672d6de43f4699f
Successfully built pyswarm
     |████████████████████████████████| 92kB 2.5MB/s 


In [2]:
from sklearn.model_selection import train_test_split
from pyswarm import pso
from os import path
import os
import requests
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
import numpy
import sys
import numpy as np
from numpy import loadtxt
from numpy import array
from numpy.random import choice
import pandas as pd
import time
import random
import statistics
import pandas
import math
import csv
import random
import logging
from pymc3 import *
import pymc3 as pm
from functools import reduce
from operator import add
from tqdm import tqdm
import geopy.distance
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
from theano import shared
from sklearn import preprocessing
print('Running on PyMC3 v{}'.format(pm.__version__))
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Activation
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import layers
from tensorflow import keras
import tensorflow
from tensorflow.keras import datasets, layers, models
from keras.utils import np_utils
from keras.utils import to_categorical

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.ticker import MaxNLocator

#TESNORFOW
import tensorflow as tf
from tensorflow import keras
from keras import datasets, layers, models

#KERAS LIBRARIES
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential,Model
from keras.layers import Dense, Dropout , Flatten,BatchNormalization,Conv2D,MaxPooling2D, Activation,LSTM,Embedding,Input,GlobalAveragePooling2D
from keras.regularizers import l1, l2, l1_l2
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend 
from keras.utils import np_utils
from keras.utils import to_categorical

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Running on PyMC3 v3.7


In [3]:
train1 = np.load('/content/drive/My Drive/NumpyArrayCovidx/train.npy',allow_pickle=True)
train_labels1 = np.load('/content/drive/My Drive/NumpyArrayCovidx/train_labels.npy',allow_pickle=True)

In [4]:
train2,test1, train_labels2,test_labels1 = train_test_split(train1, train_labels1, test_size=0.2, random_state=42)

In [5]:
train=train2/225.0
train_labels = pd.get_dummies(train_labels2)

test=test1/225.0
test_labels = pd.get_dummies(test_labels1)

In [6]:
IMG_SHAPE1=(64,64,3) # θα το προσαρμόσουμε

resnet=keras.applications.resnet_v2.ResNet152V2(input_shape=IMG_SHAPE1,
                                               include_top=False,
                                               weights='imagenet')

234553344/234545216 [==============================] - 1s 0us/step


In [7]:
#fine_tuning,flatvsglobalpool,dropout,dense_layers
lb=[0,0,0,0]
ub=[19,1,0.5,4]

In [14]:
def create_model_dense_resnet(x): 
  tempre=resnet
  for layer in tempre.layers[:(-1)*int(round(x[0]))]:
    layer.trainable = False
  
  #vgg19.trainable=False
  model = tf.keras.Sequential()
  model.add(tempre)
  if (int(round(x[1]))):
    model.add(keras.layers.Flatten())
  else:
    model.add(keras.layers.GlobalAveragePooling2D())
  if (int(round(x[3]))==4):
    layers=[256,128,64,32]
    for i in range(len(layers)):
          model.add(keras.layers.Dense(layers[i], activation='relu'))
          model.add(keras.layers.Dropout((x[2])))
  elif (int(round(x[3]))==3):
    layers=[128,64,32]
    for i in range(len(layers)):
          model.add(keras.layers.Dense(layers[i], activation='relu'))
          model.add(keras.layers.Dropout((x[2])))
  elif (int(round(x[3]))==2):
    layers=[64,32]
    for i in range(len(layers)):
          model.add(keras.layers.Dense(layers[i], activation='relu'))
          model.add(keras.layers.Dropout((x[2])))
  elif (int(round(x[3]))==1):
    layers=[32]
    for i in range(len(layers)):
          model.add(keras.layers.Dense(layers[i], activation='relu'))
          model.add(keras.layers.Dropout((x[2])))
  else:
    layers=[]
    for i in range(len(layers)):
          model.add(keras.layers.Dense(layers[i], activation='relu'))
          model.add(keras.layers.Dropout((x[2])))

  for layer in model.layers:
    print(layer, layer.trainable)
  model.add(keras.layers.Dense(3,activation="softmax"))
  model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=["accuracy"])
  return model

In [9]:
EarlyStopper = EarlyStopping(patience=4, monitor='val_loss', mode='min')

In [15]:
def apple(x):
  model = create_model_dense_resnet(x)
  model.fit(train, train_labels, epochs=20, batch_size=1000, verbose=1,validation_data=(test, test_labels),callbacks=[EarlyStopper])
  loss, _ = model.evaluate(test, test_labels, verbose=1)
  return loss 

In [16]:
xopt, fopt = pso(apple, lb, ub, swarmsize=10, omega=0.5, phip=0.5, phig=1.0, maxiter=30, minstep=1)
print ("Best position"+str(xopt))
print ("Loss:" + str(fopt))

<tensorflow.python.keras.engine.functional.Functional object at 0x7fd8da6ed358> True
<tensorflow.python.keras.layers.core.Flatten object at 0x7fd8da0bad68> True
<tensorflow.python.keras.layers.core.Dense object at 0x7fd8da0baef0> True
<tensorflow.python.keras.layers.core.Dropout object at 0x7fd8da0ba780> True
Epoch 1/20
17/17 [==============================] - 11s 632ms/step - loss: 0.6801 - accuracy: 0.7622 - val_loss: 1.1632 - val_accuracy: 0.7117
Epoch 2/20
17/17 [==============================] - 7s 405ms/step - loss: 0.2775 - accuracy: 0.8939 - val_loss: 0.8772 - val_accuracy: 0.8008
Epoch 3/20
17/17 [==============================] - 7s 404ms/step - loss: 0.1438 - accuracy: 0.9455 - val_loss: 1.3442 - val_accuracy: 0.8013
Epoch 4/20
17/17 [==============================] - 7s 404ms/step - loss: 0.0544 - accuracy: 0.9812 - val_loss: 1.5333 - val_accuracy: 0.8297
Epoch 5/20
17/17 [==============================] - 7s 403ms/step - loss: 0.0301 - accuracy: 0.9906 - val_loss: 1.6517 -